In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
# Load the data
temp = pd.read_csv('/content/drive/MyDrive/B A 데이터 모음/00클러스터링/Preprocessed_datasets/OD_PEAK.csv')

In [ ]:
temp

In [ ]:
def remove_same_origin_dest_rows(df):
    return df[df['code_origin'] != df['code_dest']]

In [ ]:
data = remove_same_origin_dest_rows(temp)

In [ ]:
data.columns

In [ ]:
data.drop(['Unnamed: 0','bus_and_subway'], axis=1, inplace= True)

In [ ]:
data.code_dest.nunique()

In [ ]:
data.shape

## 통행량 데이터

In [ ]:
outgoing_traffic = data.groupby('code_origin').sum()
incoming_traffic = data.groupby('code_dest').sum()

In [ ]:
total_traffic = outgoing_traffic.add(incoming_traffic, fill_value=0)
total_traffic

In [ ]:
total_traffic.drop(['code_origin','code_dest'], axis=1, inplace=True)
total_traffic

In [ ]:
total_traffic_sorted = total_traffic.sort_values(by='transportation_total', ascending=False)

In [ ]:
total_traffic_sorted.head()

In [ ]:
total_traffic_sorted.reset_index(inplace=True)

In [ ]:
!pip install kneed

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator

# Style
sns.set(style="whitegrid", palette="muted")

# Data sorting and find knee point
sorted_data = total_traffic_sorted['transportation_total'].sort_values(ascending=False).values
knee_locator = KneeLocator(range(len(sorted_data)), sorted_data, curve='convex', direction='decreasing')
knee_point = knee_locator.knee

# Visualization
plt.figure(figsize=(12, 7))
plt.plot(range(len(sorted_data)), sorted_data, label="Total Traffic Volume", linewidth=2)
plt.scatter(knee_point, sorted_data[knee_point], color='red', s=150, zorder=5, label='Knee Point')
plt.title("Knee Point Visualization for High Traffic Volumes", fontsize=18, fontweight='bold')
plt.xlabel("Index", fontsize=14)
plt.ylabel("Traffic Volume", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.annotate('Knee Point', xy=(knee_point, sorted_data[knee_point]), xytext=(knee_point+10, sorted_data[knee_point]+10),
             arrowprops=dict(facecolor='black', shrink=0.05), fontsize=12)

plt.legend()
plt.show()

# Knee Point 인덱스 출력
print("Knee Point Index:", knee_point)


In [ ]:
data_up_to_knee = total_traffic_sorted.iloc[:knee_point+1]

## Populatio density

In [ ]:
df2 = pd.read_csv("/content/drive/My Drive/features.csv")
df2 = df2.sort_values(by='pop_density', ascending=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator

# Style
sns.set(style="whitegrid", palette="muted")

# Data sorting and find knee point
sorted_data = df2['pop_density'].sort_values(ascending=False).values
knee_locator = KneeLocator(range(len(sorted_data)), sorted_data, curve='convex', direction='decreasing')
knee_point = knee_locator.knee

# Visualization
plt.figure(figsize=(12, 7))
plt.plot(range(len(sorted_data)), sorted_data, label="Total Traffic Volume", linewidth=2)
plt.scatter(knee_point, sorted_data[knee_point], color='red', s=150, zorder=5, label='Knee Point')
plt.title("Knee Point Visualization for Pop_Density", fontsize=18, fontweight='bold')
plt.xlabel("Index", fontsize=14)
plt.ylabel("Pop_Density", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.annotate('Knee Point', xy=(knee_point, sorted_data[knee_point]), xytext=(knee_point+10, sorted_data[knee_point]+10),
             arrowprops=dict(facecolor='black', shrink=0.05), fontsize=12)

plt.legend()
plt.show()

# Knee Point 인덱스 출력
print("Knee Point Index:", knee_point)


In [ ]:
data_up_to_knee2 = df2.iloc[:knee_point+1]
data_up_to_knee2

## # of Companies

In [ ]:
df3 = pd.read_csv("/content/drive/My Drive/features.csv")
df3 = df3.sort_values(by='company', ascending=False)

In [ ]:

# Style
sns.set(style="whitegrid", palette="muted")

# Data sorting and find knee point
sorted_data = df3['company'].sort_values(ascending=False).values
knee_locator = KneeLocator(range(len(sorted_data)), sorted_data, curve='convex', direction='decreasing')
knee_point = knee_locator.knee

# Visualization
plt.figure(figsize=(12, 7))
plt.plot(range(len(sorted_data)), sorted_data, label="Total Traffic Volume", linewidth=2)
plt.scatter(knee_point, sorted_data[knee_point], color='red', s=150, zorder=5, label='Knee Point')
plt.title("Knee Point Visualization for Number of Company", fontsize=18, fontweight='bold')
plt.xlabel("Index", fontsize=14)
plt.ylabel("Number of Company", fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.annotate('Knee Point', xy=(knee_point, sorted_data[knee_point]), xytext=(knee_point+10, sorted_data[knee_point]+10),
             arrowprops=dict(facecolor='black', shrink=0.05), fontsize=12)

plt.legend()
plt.show()

# Knee Point 인덱스 출력
print("Knee Point Index:", knee_point)


In [ ]:
# Create a new DataFrame up to the knee point
data_up_to_knee3 = df3.iloc[:knee_point+1]

data_up_to_knee3

In [ ]:
temp_unique = temp.drop_duplicates(subset='code_origin')[['code_origin', 'gu_origin', 'dong_origin']]

In [ ]:
merged_data = pd.merge(data_up_to_knee, temp_unique, on='code_origin', how='left')

In [ ]:
lldata = pd.read_csv("/content/drive/MyDrive/B A 데이터 모음/00클러스터링/위경도.csv")

In [ ]:
lldata.head()

In [ ]:
final_data = merged_data.merge(lldata[['dong', 'x', 'y']], left_on='dong_origin', right_on='dong', how='left')

In [ ]:
merged_data

In [ ]:
final_data.drop('dong', axis=1)

In [ ]:
# knee point 통일성위해서
# Select only the required columns
final_data = final_data[['transportation_total', 'gu_origin', 'dong_origin']]

# Rename the columns
final_data.rename(columns={'gu_origin': 'gu', 'dong_origin': 'dong'}, inplace=True)

# Display the modified DataFrame
final_data


In [ ]:
final_data_gu_dong

In [ ]:
data_up_to_knee2_gu_dong

In [ ]:
data_up_to_knee3_gu_dong

In [ ]:
final_data_gu_dong = final_data[['gu', 'dong']]
data_up_to_knee2_gu_dong = data_up_to_knee2[['gu', 'dong']]
data_up_to_knee3_gu_dong = data_up_to_knee3[['gu', 'dong']]

# Concatenate the DataFrames
combined_data = pd.concat([final_data_gu_dong, data_up_to_knee2_gu_dong, data_up_to_knee3_gu_dong])

# Drop duplicates
combined_data_unique = combined_data.drop_duplicates()

# Reset the index if needed
combined_data_unique.reset_index(drop=True, inplace=True)

In [ ]:
combined_data_unique

In [ ]:
combined_data_unique.to_csv("/content/drive/MyDrive/resultofkneepoints.csv")